This is a combination of:
* https://bj.lianjia.com/
* https://bj.ke.com/
* https://bj.fang.com/


Automation of collecting house for sale data and some analysis if needed

Since in Beijing new construction is pretty far we are not cosidering it now.    
Mainly focus on used house/apt around Shuangjing

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import matplotlib
import pandas_profiling

## 链家找房

In [2]:
# urllist contains all urls that should be go over
# It includes all different areas: Shuangjing, Jinsong, etc

urllist= ["https://bj.lianjia.com/ershoufang/rs%E5%8F%8C%E4%BA%95/",# Shuang jing
          "https://bj.lianjia.com/ershoufang/rs%E5%8A%B2%E6%9D%BE/" # Jin song
           ]

In [3]:
def LJ_urlmaker(urllist, nbr = 0,
             pgnbr = 1,
             price = 0,# 售价
             bedroom = 0,# 房型
             area = 0,# 面积
             age = 0,# 楼龄
             lift = 0# 电梯
            ):
    
    url = urllist[nbr]
    url1 = url.split("/")[-2]
    
    
    if price == 1:
        price_suffix = "p1" # <200
    elif price  == 2:
        price_suffix = "p2" # 200-250
    elif price  == 3:
        price_suffix = "p3" # 250-300
    elif price  == 4:
        price_suffix = "p4" # 300-400
    elif price  == 5:
        price_suffix= "p5" # 400-500
    elif price  == 6:
        price_suffix= "p6" # 500-800
    elif price  == 7:
        price_suffix= "p7" # 800-1000
    elif price  == 8:
        price_suffix= "p8" # >1000
    else:
        price_suffix =""
        
    if bedroom == 1:
        bd_suffix =  "l1"
    elif bedroom  == 2:
        bd_suffix =  "l2"
    elif bedroom  == 3:
        bd_suffix =  "l3"
    elif bedroom  == 4:
        bd_suffix =  "l4"
    elif bedroom  == 5:
        bd_suffix =  "l5"
    else:
        bd_suffix =""

    if area == 1:
        area_suffix = "a1" # <50
    elif area  == 2:
        area_suffix = "a2" # 50-70
    elif area  == 3:
        area_suffix = "a3" # 70-90
    elif area  == 4:
        area_suffix = "a4" # 90-110
    elif area  == 5:
        area_suffix= "a5" # 110-130
    elif area  == 6:
        area_suffix= "a6" # 130-150
    elif area  == 7:
        area_suffix= "a7" # 150-200
    elif area  == 8:
        area_suffix= "a8" # >200
    else:
        area_suffix =""

    if age == 1:
        age_suffix =  "y1" # <5
    elif bedroom  == 2:
        age_suffix =  "y2" # <10
    elif bedroom  == 3:
        age_suffix =  "y3" # <15
    elif bedroom  == 4:
        age_suffix =  "y4" # <20
    elif bedroom  == 5:
        age_suffix =  "y5" # >20
    else:
        age_suffix =""

    if lift == 1:
        lift_suffix =  "ie2" # has lift
    elif lift  == 2:
        lift_suffix =  "ie1" # no lift
    else:
        lift_suffix =""
    
    
    url2 = "https://bj.lianjia.com/ershoufang/" + "pg" + str(pgnbr)+ price_suffix + bd_suffix + area_suffix + age_suffix + lift_suffix + url1 + "/"
    
    return url

# a = Lianjia_urlmaker(urllist, nbr = 0,
#              pgnbr = 1,
#              price = 0,# 售价
#              bedroom = 0,# 房型
#              area = 0,# 面积
#              age = 0,# 楼龄
#              lift = 0# 电梯
#             )
# a

In [4]:
def get_url_contents(url_to_scrape):
    headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36'
            }
    
    response = requests.get(url_to_scrape,headers=headers,timeout=5)
    if response.status_code == 200:
        return response
    else:
        print("internet problem!")

# aa = get_url_contents(a)

In [5]:
def LJ_get_dataframe(response):    
    soup = BeautifulSoup(response.text, "html.parser")
    df = pd.DataFrame()
    
    comment_list = soup.find_all("div",{"class":"title"})
    location_list = soup.find_all("div",{"class":"positionInfo"})
    address_list = soup.find_all("div",{"class":"address"})
#     houseInfo_list = soup.find_all("div",{"class":"houseInfo"})
    followInfo_list = soup.find_all("div",{"class":"followInfo"})
    tag_list = soup.find_all("div",{"class":"tag"})
    priceInfo_list = soup.find_all("div",{"class":"priceInfo"})
    
    for i in range(len(location_list)):
            
#         cut address and price information
        address_info = address_list[i].text.split("|")
        price_info = priceInfo_list[i].text.split("单价")
#         get url of this apt
        url = comment_list[i].a['href']
        
        d = {
             "name" : comment_list[i].text, 
             "area" : address_info[1].split("平")[0], 
             "direction": address_info[2],
             "location":location_list[i].text,
             "floorplan": address_info[0], 
             "high/low": address_info[4], 
             "year": address_info[5].split("年")[0],
             "decoration": address_info[3],
             "type": address_info[6],
            "follow_info":followInfo_list[i].text,
            "tag": tag_list[i].text,
             "url": url,
             "total_price": price_info[0].split("万")[0],
             "unit_price": price_info[1].split("元")[0],
            "source":"链家"
            }
        df = df.append(d, ignore_index=True)
#         print("get dataframe finished!")

    return df


# LJ_get_dataframe(aa)

In [6]:
def get_info(urllist): 
    
    result_df1 = pd.DataFrame()
    for j in range(len(urllist)):
        url_1 =LJ_urlmaker(urllist, nbr = 0,
             pgnbr = 1,
             price = 0,# 售价
             bedroom = 0,# 房型
             area = 0,# 面积
             age = 0,# 楼龄
             lift = 0# 电梯
            )
        response1 = get_url_contents(url_1)
        soup = BeautifulSoup(response1.text, "html.parser")
        amount = int(soup.find("h2",{"class":"total fl"}).span.text)
        pgnbr_to = int(amount)//30+1
        print("%d pages in total" % (pgnbr_to))
             
        for i in range(pgnbr_to): 
            url_to_scrape = LJ_urlmaker(urllist, nbr = 0,
                                         pgnbr = i,
                                         price = 0,# 售价
                                         bedroom = 0,# 房型
                                         area = 0,# 面积
                                         age = 0,# 楼龄
                                         lift = 0# 电梯
                                       )
            print("scraping:"+url_to_scrape+"...")
            response = get_url_contents(url_to_scrape)    
            result_df = LJ_get_dataframe(response)
            result_df.reset_index(drop = True, inplace = True)
        
        result_df1=pd.concat([result_df,result_df1],axis=0)
        result_df1.reset_index(drop = True, inplace = True)
    
    print("get info finished!!")
    return result_df1

In [7]:
final = get_info(urllist)

32 pages in total
scraping:https://bj.lianjia.com/ershoufang/rs%E5%8F%8C%E4%BA%95/...
scraping:https://bj.lianjia.com/ershoufang/rs%E5%8F%8C%E4%BA%95/...
scraping:https://bj.lianjia.com/ershoufang/rs%E5%8F%8C%E4%BA%95/...
scraping:https://bj.lianjia.com/ershoufang/rs%E5%8F%8C%E4%BA%95/...
scraping:https://bj.lianjia.com/ershoufang/rs%E5%8F%8C%E4%BA%95/...
scraping:https://bj.lianjia.com/ershoufang/rs%E5%8F%8C%E4%BA%95/...
scraping:https://bj.lianjia.com/ershoufang/rs%E5%8F%8C%E4%BA%95/...
scraping:https://bj.lianjia.com/ershoufang/rs%E5%8F%8C%E4%BA%95/...
scraping:https://bj.lianjia.com/ershoufang/rs%E5%8F%8C%E4%BA%95/...
scraping:https://bj.lianjia.com/ershoufang/rs%E5%8F%8C%E4%BA%95/...
scraping:https://bj.lianjia.com/ershoufang/rs%E5%8F%8C%E4%BA%95/...
scraping:https://bj.lianjia.com/ershoufang/rs%E5%8F%8C%E4%BA%95/...
scraping:https://bj.lianjia.com/ershoufang/rs%E5%8F%8C%E4%BA%95/...
scraping:https://bj.lianjia.com/ershoufang/rs%E5%8F%8C%E4%BA%95/...
scraping:https://bj.lianjia.co

In [8]:
final.shape

(60, 15)

In [19]:
# final.profile_report()

## 贝壳找房

In [10]:
# seems like BK and LJ have same website structure

In [11]:
BK_urllist= ["https://bj.ke.com/ershoufang/rs%E5%8F%8C%E4%BA%95/",# Shuang jing
          "https://bj.ke.com/ershoufang/rs%E5%8A%B2%E6%9D%BE/" # Jin song
           ]

In [16]:
def BK_get_dataframe(response):    
    soup = BeautifulSoup(response.text, "html.parser")
    df = pd.DataFrame()
    
    comment_list = soup.find_all("div",{"class":"title"})
    location_list = soup.find_all("div",{"class":"positionInfo"})
#     address_list = soup.find_all("div",{"class":"address"})
    houseInfo_list = soup.find_all("div",{"class":"houseInfo"})
    followInfo_list = soup.find_all("div",{"class":"followInfo"})
    tag_list = soup.find_all("div",{"class":"tag"})
    priceInfo_list = soup.find_all("div",{"class":"priceInfo"})
    
    for i in range(len(location_list)):
            
#         cut address and price information
        address_info = houseInfo_list[0].text.replace("\n","").replace(" ","").split("|")
        price_info = priceInfo_list[i].text.replace("\n","").split("单价")
#         get url of this apt
        url = comment_list[i].a['href']
        
        d = {
             "name" : comment_list[i].text.replace("\n",""), 
             "area" : address_info[3].split("平")[0], 
             "direction": address_info[4],
             "location":location_list[i].text.replace("\n",""),
             "floorplan": address_info[2], 
             "high/low": "不适用", 
             "year": address_info[1].split("年")[0],
             "decoration": "不适用",
             "type": "不适用",
            "follow_info":followInfo_list[i].text.replace("\n",""),
            "tag": tag_list[i].text.replace("\n",""),
             "url": url,
             "total_price": price_info[0].split("万")[0],
             "unit_price": price_info[1].split("元")[0],
            "source":"贝壳"
            }
        df = df.append(d, ignore_index=True)
#         print("get dataframe finished!")

    return df


# BK_get_dataframe(aa)

In [17]:
def BK_get_info(urllist): 
    
    result_df1 = pd.DataFrame()
    for j in range(len(urllist)):
        url_1 =LJ_urlmaker(urllist, nbr = 0,
             pgnbr = 1,
             price = 0,# 售价
             bedroom = 0,# 房型
             area = 0,# 面积
             age = 0,# 楼龄
             lift = 0# 电梯
            )
        response1 = get_url_contents(url_1)
        soup = BeautifulSoup(response1.text, "html.parser")
        amount = int(soup.find("h2",{"class":"total fl"}).span.text)
        pgnbr_to = int(amount)//30+1
        print("%d pages in total" % (pgnbr_to))
             
        for i in range(pgnbr_to): 
            url_to_scrape = LJ_urlmaker(urllist, nbr = 0,
                                         pgnbr = i,
                                         price = 0,# 售价
                                         bedroom = 0,# 房型
                                         area = 0,# 面积
                                         age = 0,# 楼龄
                                         lift = 0# 电梯
                                       )
            print("scraping:"+url_to_scrape+"...")
            response = get_url_contents(url_to_scrape)    
            result_df = BK_get_dataframe(response)
            result_df.reset_index(drop = True, inplace = True)
        
        result_df1=pd.concat([result_df,result_df1],axis=0)
        result_df1.reset_index(drop = True, inplace = True)
    
    print("get info finished!!")
    return result_df1

In [18]:
final1 = BK_get_info(BK_urllist)

32 pages in total
scraping:https://bj.ke.com/ershoufang/rs%E5%8F%8C%E4%BA%95/...
scraping:https://bj.ke.com/ershoufang/rs%E5%8F%8C%E4%BA%95/...
scraping:https://bj.ke.com/ershoufang/rs%E5%8F%8C%E4%BA%95/...
scraping:https://bj.ke.com/ershoufang/rs%E5%8F%8C%E4%BA%95/...
scraping:https://bj.ke.com/ershoufang/rs%E5%8F%8C%E4%BA%95/...
scraping:https://bj.ke.com/ershoufang/rs%E5%8F%8C%E4%BA%95/...
scraping:https://bj.ke.com/ershoufang/rs%E5%8F%8C%E4%BA%95/...
scraping:https://bj.ke.com/ershoufang/rs%E5%8F%8C%E4%BA%95/...
scraping:https://bj.ke.com/ershoufang/rs%E5%8F%8C%E4%BA%95/...
scraping:https://bj.ke.com/ershoufang/rs%E5%8F%8C%E4%BA%95/...
scraping:https://bj.ke.com/ershoufang/rs%E5%8F%8C%E4%BA%95/...
scraping:https://bj.ke.com/ershoufang/rs%E5%8F%8C%E4%BA%95/...
scraping:https://bj.ke.com/ershoufang/rs%E5%8F%8C%E4%BA%95/...
scraping:https://bj.ke.com/ershoufang/rs%E5%8F%8C%E4%BA%95/...
scraping:https://bj.ke.com/ershoufang/rs%E5%8F%8C%E4%BA%95/...
scraping:https://bj.ke.com/ershoufang

In [20]:
final1.shape

(60, 15)